# Praktikum Pengantar Pembelajaran Mesin


---
## Bab 6. Support Vector Machine (SVM)


Tuliskan Nama, NIM, dan kelas Anda:

Nama  : Dani Adrian

NIM   : 225150201111009

Kelas : TIF-E

### 1) Import Data

Unduh dataset yang akan digunakan pada praktikum kali ini. Anda dapat menggunakan aplikasi wget untuk mendowload dataset dan menyimpannya dalam Google Colab. Jalankan cell di bawah ini untuk mengunduh dataset

In [ ]:
!wget https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv

--2024-04-28 07:04:59--  https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3975 (3.9K) [text/plain]
Saving to: ‘iris.csv.13’

iris.csv.13         100%[===================>]   3.88K  --.-KB/s    in 0s      

2024-04-28 07:04:59 (57.4 MB/s) - ‘iris.csv.13’ saved [3975/3975]



Setelah dataset berhasil diunduh, langkah berikutnya adalah membaca dataset dengan memanfaatkan fungsi **readcsv** dari library pandas. Lakukan pembacaan berkas csv ke dalam dataframe dengan nama **data** menggunakan fungsi **readcsv**. Jangan lupa untuk melakukan import library pandas terlebih dahulu


In [ ]:
import pandas as pd
import numpy as np
data = pd.read_csv('iris.csv')



Cek isi dataset Anda dengan menggunakan perintah **head()**

In [ ]:
data.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


Metode Support Vector Machine (SVM) dasar hanya mampu melakukan klasifikasi data yang terdiri dari dua kelas, atau disebut klasifikasi biner. Dataset iris memiliki 3 kelas, sehingga salah satu kelasnya, yaitu Iris-virginica perlu dihapus. Cell berikut menghapus data yang memiliki kelas Iris-virginica

In [ ]:
data.drop(data[data['variety']=='Iris-virginica'].index,inplace=True)

SVM juga memiliki keterbatasan, yaitu hanya mampu menerima kelas dalam bentuk numerik. Cell berikut mengubah kelas menjadi bilangan, Iris setosa menjadi -1 dan Iris-versicolor menjadi 1

In [ ]:
data['variety']=data['variety'].map({'Setosa':-1,'Versicolor':1})

Tampilkan beberapa data untuk mengecek hasilnya

In [ ]:
data.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,-1.0
1,4.9,3.0,1.4,0.2,-1.0
2,4.7,3.2,1.3,0.2,-1.0
3,4.6,3.1,1.5,0.2,-1.0
4,5.0,3.6,1.4,0.2,-1.0


## 2) Membagi data menjadi data latih dan data uji

Metode pembelajaran mesin memerlukan dua jenis data :


1.   Data latih : Digunakan untuk proses training metode klasifikasi
2.   Data uji : Digunakan untuk proses evaluasi metode klasifikasi

Data uji dan data latih perlu dibuat terpisah (mutualy exclusive) agar hasil evaluasi lebih akurat.

Data uji dan data latih dapat dibuat dengan cara membagi dataset dengan rasio tertentu, misalnya 80% data latih dan 20% data uji.

Library Scikit-learn memiliki fungsi [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) pada modul **model_selection** untuk membagi dataset menjadi data latih dan data uji. Bagilah dataset anda menjadi dua, yaitu **data_latih** dan **data_uji**.


In [ ]:
from sklearn.model_selection import train_test_split
data_latih,data_uji = train_test_split(data,test_size=0.2)

Tampilkan banyaknya data pada **data_latih** dan **data_uji**. Seharusnya **data_latih** terdiri dari 80 data, dan **data_uji** terdiri dari 20 data

In [ ]:
print(data_latih.shape[0])
print(data_uji.shape[0])

120
30


Pisahkan label/kategori dari data latih dan data uji menjadi variabel tersendiri. Beri nama variabelnya  **label_latih** dan **label_uji**

In [ ]:
label_latih = data_latih.pop('variety')

In [ ]:
label_uji = data_uji.pop('variety')

## 3) Proses Training

Tujuan dari algoritma SVM adalah meminimalkan nilai *cost function*. Penghitungan nilai minimal dapat dapat dilakukan dengan menghitung nilai gradien dari *cost function* terlebih dahulu. Fungsi di bawah ini berguna untuk menghitung nilai gradien cost function

In [ ]:
def hitung_cost_gradient(W,X,Y,regularization):
    jarak = 1 - (Y* np.dot(X,W))
    dw = np.zeros(len(W))
    if max(0,jarak)==0:
      di=W
    else:
      di = W - (regularization * Y*X)
    dw += di
    return dw

Terdapat beberapa cara untuk meminimalkan nilai *cost function*, salah satunya menggunakan Stochastic Gradient Descent (SGD) untuk melakukan minimasi. Minimasi *cost function* merupakan inti dari algoritma SVM. Fungsi di bawah ini merupakan implementasi algoritma SGD

In [ ]:
from sklearn.utils import shuffle
def sgd(data_latih,label_latih,learning_rate = 0.000001,max_epoch=1000,regularization=10000):
  data_latih = data_latih.to_numpy()
  label_latih = label_latih.to_numpy()
  bobot = np.zeros(data_latih.shape[1])
  for epoch in range(1,max_epoch):
    X,Y = shuffle(data_latih,label_latih,random_state=101)
    for index,x in enumerate(X):
      delta = hitung_cost_gradient(bobot,x,Y[index],regularization)
      bobot = bobot - (learning_rate * delta)
  return bobot

Proses training dilakukan dengan memanggil fungsi **sgd** dengan parameter input berupa data latih dan label latih. Parameter learning rate dan max epoch menggunakan nilai *default* nya

In [ ]:
W = sgd(data_latih,label_latih)
print(W)

[-0.22770987 -0.63032056  1.00373556  0.42677103]


## 4) Proses *testing*
Proses testing dilakukan dengan menghitung nilai [*dot product*](https://en.wikipedia.org/wiki/Dot_product) antara bobot hasil training dengan data uji. Kelas data ditentukan berdasarkan tanda (positif atau negatif) dari hasil dot product tersebut. Fungsi berikut mengimplementasikan proses testing

In [ ]:
def testing(W,data_uji):
  prediksi = np.array([])
  for i in range(data_uji.shape[0]):
    y_prediksi = np.sign(np.dot(W,data_uji.to_numpy()[i]))
    prediksi = np.append(prediksi,y_prediksi)
  return prediksi

Lakukan pengujian menggunakan seluruh data uji. Bandingkan hasilnya dengan nilai label sebenarnya untuk menghitung berapa banyak data uji yang berhasil diprediksi dengan benar.

In [ ]:
y_prediksi = testing(W,data_uji)
print(sum(y_prediksi==label_uji))

21


## TUGAS
Pada tugas kali ini Anda masih melakukan klasifikasi dataset iris, tetapi menggunakan kelas Iris-versicolor dan Iris-virginica. Apakah hasilnya lebih baik jika dibandingkan dengan Iris-setosa dan Iris-versicolor?

**Note: Tambahkan blok kode apabila diperlukan**

In [ ]:
data2 = pd.read_csv('iris.csv')

In [ ]:
data2.drop(data2[data2['variety']=='Setosa'].index,inplace=True)
data2['variety']=data2['variety'].map({'Versicolor':-1,'Virginica':1,})
data_latih2, data_uji2 = train_test_split(data2,test_size=0.2)

In [ ]:
label_latih2 = data_latih2.pop('variety')
label_uji2 = data_uji2.pop('variety')

In [ ]:
W2 = sgd(data_latih2, label_latih2)
print(W2)

[-4.84913871 -1.77278019  4.83803949  6.76830101]


In [ ]:
y_prediksi2 = testing(W2,data_uji2)
print(sum(y_prediksi2==label_uji2))

18


Jawaban:
Hasilnya tidak lebih baik karena hanya mampu mengenali 18 diantara 20 data uji. Meskipun begitu, hasil ini masih dapat dibilang cukup akurat.

Lakukan proses training ulang, tetapi menggunakan nilai max_epoch sebesar 5000. Anda dapat memberikan parameter max_epoch=5000 saat memanggil fungsi **sgd**. Apakah hasilnya lebih baik? Mengapa?

In [ ]:
W2 = sgd(data_latih2,label_latih2,max_epoch = 5000)
print(W2)

[-5.43729593 -2.08086572  5.64639975  7.30652309]


In [ ]:
y_prediksi2 = testing(W2,data_uji2)
print(sum(y_prediksi2==label_uji2))

18


Jawaban:

Akurasi hasil tetap stabil karena perubahan nilai max epoch hanya membandingkan kinerja model pada data latih dan uji, tanpa memengaruhi hasilnya. Meskipun max epoch diperbesar, hasil tetap konsisten. Namun, peningkatan nilai max epoch juga berarti penambahan waktu kompilasi yang lebih lama.

Terakhir, lakukanlah seleksi fitur sederahana dengan mengambil hanya fitur sepal length dan sepal width saja. Apakah hasilnya lebih baik?

In [ ]:
data3 = data2.drop(['petal.length', 'petal.width'],axis=1)
data_latih3, data_uji3 = train_test_split(data3,test_size=0.2)
label_latih3= data_latih3.pop('variety')
label_uji3 = data_uji3.pop('variety')

In [ ]:
W3 = sgd(data_latih3, label_latih3, max_epoch=5000)
print(W3)

[ 0.5694657 -1.1345491]


In [ ]:
y_prediksi_tugas = testing(W3,data_uji3)
print(sum(y_prediksi_tugas==label_uji3))

11


Jawaban:

Kualitas tidak meningkat, malah menurun dan menjadi kurang akurat karena jumlah variabel yang digunakan untuk prediksi berkurang. Ini terjadi karena fitur petal.length dan petal.width telah dihapus. Akibatnya, informasi yang tersedia menjadi lebih terbatas, membuat klasifikasi antarkelas menjadi lebih sulit karena kurangnya keunikannya.